# LLM 101 PoC, Tester

Goal: Test implementations interatively.

# Library

In [1]:
import sys  
sys.path.insert(1, '/home/jovyan/workspace/src')

In [2]:
import utils
import controller
import importlib
importlib.reload(utils)
importlib.reload(controller)

<module 'controller' from '/home/jovyan/workspace/src/controller.py'>

In [3]:
tool_utils = utils.UtilsLLM()

2024-06-07 20:45:02,299 - root - INFO - Utils initialized


In [4]:
tool_controller = controller.Controller('https://staffeng.com/guides/staff-archetypes/', 'what is the solver role?')

2024-06-07 20:45:02,582 - root - INFO - Utils initialized
2024-06-07 20:45:02,688 - root - INFO - Controller initialized


# Test

In [5]:
# Auth
tool_controller.authenticate()

2024-06-07 20:45:02,796 - root - INFO - Env vars loaded: False
2024-06-07 20:45:02,820 - root - INFO - GOOGLE_API_KEY len: 39
2024-06-07 20:45:02,869 - root - INFO - PINECONE_API_KEY len: 36
2024-06-07 20:45:02,878 - root - INFO - OPENAI_API_KEY len: 56
2024-06-07 20:45:02,885 - root - INFO - Succeed authenticating.


(True, 'Succeed authenticating.')

In [6]:
# Define embedding
tool_controller.define_embedding_model()

2024-06-07 20:45:02,950 - root - INFO - Succeed defining the embedding model
2024-06-07 20:45:03,227 - root - DEBUG - Embedding model defined: model='models/embedding-001' task_type=None google_api_key=None credentials=None client_options=None transport=None request_options=None


(GoogleGenerativeAIEmbeddings(model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None),
 'Succeed defining the embedding model')

In [7]:
# --- Prepare content
# Check existence of Pineocone index
tool_controller.check_if_vector_store_index_already_exists_for_this_url('https://staffeng.com/guides/staff-archetypes/')

2024-06-07 20:45:03,255 - root - DEBUG - Checking URL https://staffeng.com/guides/staff-archetypes/ hashed to 2aa286980436145bc4d29577d69b9c69
2024-06-07 20:45:03,261 - root - DEBUG - Looking for 2aa286980436145bc4d29577d69b9c69
2024-06-07 20:45:03,634 - root - DEBUG - ... in ['2aa286980436145bc4d29577d69b9c69']
2024-06-07 20:45:04,456 - root - INFO - Specific index name exists in Pinecone
2024-06-07 20:45:04,464 - root - INFO - Does a vector store index already exist for this URL: True


True

In [8]:
# Check url value
print(tool_controller.content_ref)

https://staffeng.com/guides/staff-archetypes/


In [9]:
# Prepare vector store client
tool_controller.vector_store_loaded_client = tool_utils.set_vector_store_client_to_specific_index(tool_controller.content_ref, tool_controller.embedding_model)

2024-06-07 20:45:07,444 - root - INFO - Succeed setting Pinecone client to a specific index


In [10]:
# Check client loaded vector store
print(tool_controller.vector_store_loaded_client)

In [11]:
# --- Prepare LLM chain
# Check if chain is ready
tool_controller.chain_is_prepared

False

In [12]:
# Define model choice
tool_controller.model_choice = "Gemini"

In [13]:
# Prepare chain
tool_controller.prepare_llm_chain("retrieved_documents", tool_controller.vector_store_loaded_client)

2024-06-07 20:45:07,600 - root - INFO - LLM model defined as Gemini
2024-06-07 20:45:07,621 - root - INFO - Succeed defining the llm model
2024-06-07 20:45:07,631 - root - INFO - Succeed preparing prompt
2024-06-07 20:45:07,644 - root - INFO - Succeed building chain with documents retrieval


In [14]:
# --- Ask question about the content
tool_controller.question = 'what is the solver role?'

In [15]:
# Ask to LLM
tool_controller.ask_question_to_llm_using_vector_store(tool_controller.model_choice, "URL")

2024-06-07 20:45:07,725 - root - INFO - LLM model defined as Gemini
2024-06-07 20:45:07,750 - root - INFO - Succeed defining the llm model
2024-06-07 20:45:07,759 - root - INFO - Succeed preparing prompt
2024-06-07 20:45:07,779 - root - INFO - Succeed building chain with documents retrieval
2024-06-07 20:45:07,787 - root - INFO - Succeed resetting chain for model: Gemini.
2024-06-07 20:45:12,928 - root - INFO - Succeed asking question about content using a retrieved documents chain
2024-06-07 20:45:12,932 - root - INFO - Question asked to Gemini
2024-06-07 20:45:12,942 - root - INFO - Q: what is the solver role?
2024-06-07 20:45:12,957 - root - INFO - A: The Solver digs deep into arbitrarily complex problems and finds an appropriate path forward. Some focus on a given area for long periods. Others bounce from hotspot to hotspot as guided by organizational leadership.


(True,
 {'input': 'what is the solver role?',
  'context': [Document(page_content="The Solver is most common in companies that think of individuals, rather than teams, as the atomic unit of planning and ownership.\nIn such companies, it's common to see the Solver become prevalent in the place of the Tech Lead.\nYou're less likely to encounter this role at traditionally managed sprint-centric companies until those companies become\nrelatively large or long-lived enough to acquire their own varietal of technical debt.\nRight Hand"),
   Document(page_content="Influential architects dedicate their energy to maintaining an intimate understanding of the business' needs, their users' goals, and the relevant technical constraints.\nThey use that insight to identify and advocate for effective approaches within their area of focus,\nand do it with organizational authority that they've earned by demonstrating consistently good judgment.\nThe Architect role tends to evolve in relatively large comp

---
# End.